# Linear Regression (Advanced Example)

Objective: Basically what we do here is examine a dataset with E-commerce Customer Data for a company's website and mobile application. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

In [1]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('linear_regression_adv').getOrCreate()

# If you're getting an error with numpy, please type 'sudo pip install numpy --user' into the EC2 console.
from pyspark.ml.regression import LinearRegression

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/12 05:28:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Use Spark to read in the Ecommerce Customers csv file. You can infer csv schemas. 
data = spark.read.csv("Datasets/ecommerce_data.csv",inferSchema=True,header=True)

In [3]:
# Print the schema of the DataFrame. You can see potential features as well as the predictor.
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [4]:
# Let's focus on one row to make it easier to read.
data.select("Email", "Time on App",'Yearly Amount Spent').show() 

+--------------------+------------------+-------------------+
|               Email|       Time on App|Yearly Amount Spent|
+--------------------+------------------+-------------------+
|mstephenson@ferna...| 12.65565114916675|  587.9510539684005|
|   hduke@hotmail.com|11.109460728682564|  392.2049334443264|
|    pallen@yahoo.com|11.330278057777512| 487.54750486747207|
|riverarebecca@gma...|13.717513665142507|  581.8523440352177|
|mstephens@davidso...|12.795188551078114|  599.4060920457634|
|alvareznancy@luca...|12.026925339755056|   637.102447915074|
|katherine20@yahoo...|11.366348309710526|  521.5721747578274|
|  awatkins@yahoo.com| 12.35195897300293|  549.9041461052942|
|vchurch@walter-ma...|13.386235275676436|  570.2004089636196|
|    bonnie69@lin.biz|11.814128294972196|  427.1993848953282|
|andrew06@peterson...|13.338975447662113|  492.6060127179966|
|ryanwerner@freema...|11.584782999535266|  522.3374046069357|
|   knelson@gmail.com|10.961298400154098|  408.6403510726275|
|wrightp

In [5]:
# A simple for loop allows us to make it even clearer. 
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up a DataFrame for Machine Learning (MLlib)

We need to do a few things before Spark can accept the data for machine learning. First of all, it needs to be in the form of two columns: label and features. Unlike the documentation example, this data is messy. We'll need to combine all of the features into a single vector. VectorAssembler simplifies the process.

In [6]:
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [7]:
# The input columns are the feature column names, and the output column is what you'd like the new column to be named. 
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [8]:
# Now that we've created the assembler variable, let's actually transform the data.

output = assembler.transform(data)

In [9]:
# Using print schema, you see that the features output column has been added. 
output.printSchema()

# You can see that the features column is a dense vector that combines the various features as expected.
output.head(1)

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [10]:
# Let's select two columns (the feature and predictor).
# This is now in the appropriate format to be processed by Spark.
final_data = output.select("features",'Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [11]:
# Let's do a randomised 70/30 split. 
# Remember, you can use other splits depending on how easy/difficult it is to train your model.
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [12]:
# Let's see our training data.
train_data.select('Yearly Amount Spent').describe().show()

test_data.select('Yearly Amount Spent').describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                355|
|   mean| 498.62798066404264|
| stddev|  80.32727248961223|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                145|
|   mean|  500.9936965073127|
| stddev|   77.0270357762301|
|    min|  304.1355915788555|
|    max|  765.5184619388373|
+-------+-------------------+



Now we can create a Linear Regression Model object. Because the feature column is named 'features', we don't have to worry about it. However, as the labelCol isn't the default name, we have to specify it's name (Yearly Amount Spent).

In [13]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [14]:
# Fit the model to the data.
lrModel = lr.fit(train_data)

24/05/12 05:28:23 WARN Instrumentation: [27eeb1ec] regParam is zero, which might cause numerical instability and overfitting.
24/05/12 05:28:23 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/05/12 05:28:23 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
24/05/12 05:28:23 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [15]:
# Print the coefficients and intercept for linear regression.
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.476929998079843,38.569837110224526,0.3412472909113483,61.53473628611583] Intercept: -1037.7618453378527


In [16]:
# Let's evaluate the model against the test data.
test_results = lrModel.evaluate(test_data)

In [17]:
# Interesting results! This shows the difference between the predicted value and the test data.
test_results.residuals.show()

# Let's get some evaluation metrics (as discussed in the previous linear regression notebook).
print("RSME: {}".format(test_results.rootMeanSquaredError))

/home/ubuntu/spark-3.2.1-bin-hadoop2.7/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-------------------+
|          residuals|
+-------------------+
|  9.465947625470108|
| -4.186402604895818|
| -6.639450764173773|
|-13.496240788574994|
|-22.030595249217754|
|  -7.62341501340029|
| 18.539774691646528|
|0.08896502410016183|
| -5.512944973673939|
|-1.4280451980904445|
|  -2.24454907198691|
| -6.420732858123472|
|-11.610789694009554|
| -17.75555161591234|
| -9.561218029461713|
|-1.8846808472088696|
|  -9.42630222413203|
| 11.173714311834487|
|  3.478198637044841|
|  0.873755235616045|
+-------------------+
only showing top 20 rows

RSME: 9.870320466381022


In [18]:
# We can also get the R2 value. 
print("R2: {}".format(test_results.r2))

R2: 0.983465858635567


Looking at RMSE and R2, we can see that the model is quite accurate. The RMSE shows that, on average, there's only a \\$10 discrepancy between the actual and predicted results. Comparing this to the table below, the average amount spent (\\$499) and standard deviation (\\$79), a \\$10 error is surprisingly good. 

The R2 also shows that the model accounts for 98% of the variance in the data. 

In [19]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



## But what if we didn't have the predictor data?

This isn't really relevant to your assignment, but useful in a real-world scenario. What if you have all of these features but no predictor data? How do you actually use the model you've created? Check out the example below.

In [20]:
# Let's just select the features column (removing the label column).
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.8794843441274...|
|[31.0613251567161...|
|[31.0662181616375...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.3123495994443...|
|[31.3895854806643...|
|[31.5147378578019...|
|[31.6610498227460...|
|[31.7216523605090...|
|[31.7242025238451...|
|[31.8093003166791...|
|[31.8164283341993...|
|[31.8279790554652...|
|[31.8627411090001...|
|[31.8854062999117...|
|[31.9096268275227...|
|[32.0123007682454...|
|[32.0444861274404...|
+--------------------+
only showing top 20 rows



In [21]:
# Now we can transform the unlabeled data.
predictions = lrModel.transform(unlabeled_data)

In [22]:
# It worked! Feeding the unlabeled data features into the model results in a prediction, 
# which is the amount someone with those features is likely to spend in a year.
predictions.show()
predictions.head(1)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 399.1744034471574|
|[30.8794843441274...| 494.3930025897505|
|[31.0613251567161...| 494.1949088220754|
|[31.0662181616375...|462.42953399624935|
|[31.1239743499119...| 508.9776490889835|
|[31.1280900496166...|  564.876101760455|
|[31.3123495994443...| 445.0516433362941|
|[31.3895854806643...| 409.9806460358827|
|[31.5147378578019...|495.32543297013535|
|[31.6610498227460...| 417.7863987779913|
|[31.7216523605090...|350.02147570385955|
|[31.7242025238451...|509.80862014608397|
|[31.8093003166791...| 548.3826890568507|
|[31.8164283341993...| 518.8780431195687|
|[31.8279790554652...|449.56396557640323|
|[31.8627411090001...| 558.1828220212556|
|[31.8854062999117...|399.52957519660754|
|[31.9096268275227...| 552.2723213614047|
|[32.0123007682454...| 489.4668544289134|
|[32.0444861274404...|447.35607395093393|
+--------------------+------------

[Row(features=DenseVector([29.5324, 10.9613, 37.4202, 4.0464]), prediction=399.1744034471574)]